In [ ]:


def preprocess_sensor(dat):
    """ 
    Sorts sensor data by timestamp and removes duplicates
    Params: 
        dat: sensor data 
    """

    dat = np.array(dat).astype(float)
    sorted_dat = dat[dat[:,0].argsort()]

    _, unique_indices = np.unique(sorted_dat[:,0], return_index = True)
    processed_data = sorted_dat[unique_indices]
    return processed_data

def clip_xl(xl_dat, clip):
    """
    XL sensors above certain thresholds are defective
    """
    xl_clipped = np.where(xl_dat > clip, clip, xl_dat)
    xl_clipped = np.where(xl_clipped < -1*clip, -1*clip, xl_clipped)
    return xl_clipped

def preprocess_imu(xl_raw, gyr_raw, gyr_offset):
    """
    correct units, remove gyr offset
    Returns:
        data_matrix: (xl, mag, gyr)
    """

    xl = xl_raw * 9.810665
    gyr = gyr_offset_radians(gyr_raw, gyr_offset)
    return xl, gyr

def gyr_offset_radians(gyr_dat, offset):
    """Put gy data in radians, remove measured error """
    gyr_off = gyr_dat - offset
    gyro_rad = gyr_off * ( np.pi / 180)
    return gyro_rad

def time_align(xyz, sensor_odr, interval_odr):
    """ 
    Upsample data 2x interval odr(fourier signal), decimate data back down(lowpass filter), interpolate to desired odr, select current interval points
    """
    xyz_w = []
    len_w = []
    for i, ts in enumerate(xyz):
        xyz_w.append(ts)
        len_w.append(len(ts))

    xyz_w = np.vstack(xyz_w)
    # if name == 'xl':
        # print('raw',xyz_w.shape)

    xyz_r = resample(xyz_w, num=(sensor_odr*3)*2, axis=0)
    factor = xyz_r.shape[0]//((sensor_odr*3))
    xyz_d = decimate(xyz_r, q=factor, axis=0)

    current_points = xyz_d[interval_odr:interval_odr*2]
    
    # print('final', np.array(current_points).shape)
    return np.array(current_points)

def normalize(q): 
    #norm = np.sqrt(q @ q.T)
    norm = magnitude(q)
    return q/norm 

def quatConjugate(q):
        w,x,y,z = q
        q_star = np.array([w,-x,-y,-z])
        return q_star
        
def quatProduct(q, p):
        q = np.array(q)
        p = np.array(p)
        w, x, y, z = list(q)

        q_matrix = np.array([
            [w, -x, -y, -z],
            [x, w, -z, y],
            [y, z, w, -x],
            [z, -y, x, w]
        ])
        return q_matrix @ p

def inverse(q):
    q_star = quatConjugate(q)
    mag = magnitude(q)
    if  0.998 < mag < 1.001:
        return q_star
    else:
        q_inverse = q_star / mag**2
        return q_inverse
    
def magnitude(q):
    return np.sqrt(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2)

def rotate_vector(q, v, active=True):
    v = np.insert(np.array(v), 0, 0)  # convert to pure quaternion
    q_inv = inverse(q)

    if active: # object rotation
        q_product = quatProduct(q, v)
        q_rot = quatProduct(q_product, q_inv)
    else: # coordinate rotation
        q_product = quatProduct(q_inv, v)
        q_rot = quatProduct(q_product, q)

    return q_rot[1:]

def R(q_w, q_x, q_y, q_z):
    """
    Direction Cosine Matrix to rotate between two frames
    """
    # rotation_matrix = np.array([
    #     [q_w**2+q_x**2-q_y**2-q_z**2, 2*(q_x*q_y-q_w*q_z), 2*(q_x*q_z+q_w*q_y)],
    #     [2*(q_x*q_y+q_w*q_z), q_w**2-q_x**2+q_y**2-q_z**2, 2*(q_y*q_z-q_w*q_x)],
    #     [2*(q_x*q_z-q_w*q_y), 2*(q_w*q_x+q_y*q_z), q_w**2-q_x**2-q_y**2+q_z**2]
    # ])
            
    rotation_matrix = np.array([
        [1.0-2.0*(q_y**2+q_z**2), 2.0*(q_x*q_y-q_w*q_z), 2.0*(q_x*q_z+q_w*q_y)],
        [2.0*(q_x*q_y+q_w*q_z), 1.0-2.0*(q_x**2+q_z**2), 2.0*(q_y*q_z-q_w*q_x)],
        [2.0*(q_x*q_z-q_w*q_y), 2.0*(q_w*q_x+q_y*q_z), 1.0-2.0*(q_x**2+q_y**2)]
        ])
    return rotation_matrix

def rotate_to_ned(q,vec):
    """
    rotates raw data by quaternion into global frame
    """
    rotation_matrix = R(*q)
    global_estimate = rotation_matrix.T @ vec  #eq. 10 
    
    return global_estimate

def quaternion_to_euler(q_L_G):
    """Calculate yaw pitch and roll from quaterion
    Args:
        q_L_G: quaternion representing rotation from local to NED frame 
    Returns:
        yaw, pitch, roll in numpy arrays
    """
    yaws, pitches, rolls = [], [], []
    for q in q_L_G:

        w, x, y, z = q
        roll = math.atan2(2*(w*x + y*z), 1 - 2*(x**2 + y**2))
        pitch = -np.pi + 2*(math.atan2(np.sqrt(1+2*(w*y-x*z)),np.sqrt(1-2*(w*y-x*z))))
        yaw = math.atan2(2*(w*z + x*y), 1 - 2*(y**2 + z**2))
        yaw, pitch, roll = np.degrees(yaw), np.degrees(pitch), np.degrees(roll)

        yaws.append(yaw)
        rolls.append(roll)
        pitches.append(pitch)
    yaws = np.array(yaws) % 360

    return yaws, np.array(pitches), np.array(rolls)

def align_timestamps(xl, gy, interval_odr, gyr_offset):
    cols =['interval','timestamp','x', 'y', 'z']

    xl_df = pd.DataFrame(xl, columns=cols)
    xl_df = xl_df.astype({'x': float, 'y': float, 'z': float, 'timestamp':float})
    xl_group = xl_df.groupby('interval')

    gy_df = pd.DataFrame(gy, columns=cols)
    gy_df = gy_df.astype({'x': float, 'y': float, 'z': float, 'timestamp':float})
    gy_group = gy_df.groupby('interval')
    
    common_ids = np.intersect1d(np.array(list(xl_group.groups.keys())), np.array(list(gy_group.groups.keys())))

    orientation = []


    xl_global = []
    prev_q = [1,0,0,0]

    xl_w = [] #(3, n, 3) history of 3, n data , xyz 
    gy_w = []

    common_ids = np.intersect1d(np.array(list(xl_group.groups.keys())), np.array(list(gy_group.groups.keys())))

    orientation = []
    for interval_id in common_ids:
        xl_grp = xl_group.get_group(interval_id).copy()

        gy_grp = gy_group.get_group(interval_id).copy()

        group_ts = [xl_grp.iloc[xl_grp.shape[0]//2]['timestamp'] for _ in range(interval_odr)]
        group_interval = [xl_grp.iloc[xl_grp.shape[0]//2]['interval'] for _ in range(interval_odr)]
        

        xl_1 = preprocess_sensor(xl_grp.drop(columns=['interval']).values)
        xl1_clipped = clip_xl(xl_1[:,1:], 2)

        gy_1 = preprocess_sensor(gy_grp.drop(columns=['interval']).values) #remove unique values 
        xl_p, gy_p = preprocess_imu(xl1_clipped, gy_1[:,1:], gyr_offset)


        xl_w.append(xl_p.copy())
        gy_w.append(gy_p.copy())

        if len(xl_w) > 3: 
            xl_w.pop(0)
            gy_w.pop(0)

        if len(xl_w) != 3:
            continue 

        xl_t = time_align(xl_w, 1000, interval_odr)
        gy_t = time_align(gy_w, 1000, interval_odr)


        model = Madgwick(acc=xl_t, gyr=gy_t, dt=1/interval_odr, q0 = [1,0,0,0], gain =.0033 )
        
        prev_q = model.Q[-1]
        interval_q = model.Q

        group_xl_ned = []
        for q,x in zip(np.array(interval_q), xl_t):
           
            mag = np.linalg.norm(x)
            norm_vec = x / mag

            ned_vec_1 = rotate_to_ned(q, norm_vec)
            vec1 = np.round(ned_vec_1,4)

            assert np.issubdtype(ned_vec_1.dtype, np.floating)
            
            ned_vec = vec1 * mag
            print(ned_vec)
            ned_vec[2] -= 9.810665

            print(ned_vec, x)
            group_xl_ned.append(ned_vec)

        assert np.issubdtype(np.array(group_xl_ned).dtype, np.floating)
        group_xl_ned = np.column_stack([group_xl_ned, group_ts])
        xl_global.extend(group_xl_ned)

        yaws, pitches, rolls = quaternion_to_euler(model.Q)
        rpy = [rolls, pitches, yaws]
        orientation.extend(rpy)
        np.array(orientation)

    return np.array(xl_global), np.array(orientation)

In [ ]:
gy_mask_combined = np.logical_and(gy_mask_x, gy_mask_y, gy_mask_z)
gy_timestamps = imu_dic["Node[1].GY1.timestamp"]
new_gy_ts = gy_timestamps[gy_mask_combined]


xl_mask_combined = np.logical_and(mask_x, mask_y, mask_z)
timestamps = imu_dic["Node[1].XL1.timestamp"]
new_xl_ts = timestamps[xl_mask_combined]

print(new_xl_ts.shape)

xl_stack = np.array([imu_dic['Node[1].XL1.interval_id'][xl_mask_combined],imu_dic['Node[1].XL1.timestamp'][xl_mask_combined], 
            imu_dic['Node[1].XL1.X'][xl_mask_combined],imu_dic['Node[1].XL1.Y'][xl_mask_combined],
              imu_dic['Node[1].XL1.Z'][xl_mask_combined]]).T

gy_stack = np.array([imu_dic['Node[1].GY1.interval_id'][gy_mask_combined],imu_dic['Node[1].GY1.timestamp'][gy_mask_combined], 
            imu_dic['Node[1].GY1.X'][gy_mask_combined],
            imu_dic['Node[1].GY1.Y'][gy_mask_combined], imu_dic['Node[1].GY1.Z'][gy_mask_combined]]).T

print(xl_stack.T.shape, gy_stack.T.shape)

In [ ]:
offset_x = np.mean(imu_dic['Node[1].GY1.X'][gy_mask_combined])
offset_y = np.mean(imu_dic['Node[1].GY1.Y'][gy_mask_combined])
offset_z = np.mean(imu_dic['Node[1].GY1.Z'][gy_mask_combined])

offset = np.array([offset_x, offset_y, offset_z])
xl_global, orientation = align_timestamps(xl_stack, gy_stack, 1000, offset)

In [ ]:
def xl_resample(xl,gy, interval_odr):
    cols =['interval','timestamp','x', 'y', 'z']

    xl_df = pd.DataFrame(xl, columns=cols)
    xl_df = xl_df.astype({'x': float, 'y': float, 'z': float, 'timestamp':float})
    xl_group = xl_df.groupby('interval')

    gy_df = pd.DataFrame(gy, columns=cols)
    gy_df = gy_df.astype({'x': float, 'y': float, 'z': float, 'timestamp':float})
    gy_group = gy_df.groupby('interval')
    
    common_ids = np.intersect1d(np.array(list(xl_group.groups.keys())), np.array(list(gy_group.groups.keys())))
    xl_resample = []
    xl_w = [] #(3, n, 3) history of 3, n data , xyz 

    common_ids = np.intersect1d(np.array(list(xl_group.groups.keys())), np.array(list(gy_group.groups.keys())))

    for interval_id in common_ids:

        xl_grp = xl_group.get_group(interval_id).copy()
        group_ts = [xl_grp.iloc[xl_grp.shape[0]//2]['timestamp'] for _ in range(interval_odr)]

        xl_1 = preprocess_sensor(xl_grp.drop(columns=['interval']).values)
        xl1_clipped = clip_xl(xl_1[:,1:], 2)
        xl_w.append(xl1_clipped.copy())

        xyz_r = resample(xyz_w, num=(interval_odr*3), axis=0)
        curr_xyz = xyz_r[1000:2000,:]

        curr_dat = np.column_stack([curr_xyz, group_ts])

        xl_resample.extend(curr_dat)

    return np.array(xl_resample)


def compute_aligned_timestamps(xl_data):

    xl_df= pd.DataFrame(xl_data, columns=['xl.x','xl.y','xl.z','timestamp'])

    new_ts_col = []
    xl_dt = 1/1000
    for ts in np.unique(xl_df['timestamp'].astype(int)):

        curr_ts = ts
        for _ in range(1000):
            new_ts_col.append(curr_ts)
            curr_ts += xl_dt

    xl_df['timestamp_aligned'] = new_ts_col
    return xl_df


In [ ]:
def rotation_ned(lat, lon):
    R_ned = np.array([[-np.sin(lat)*np.cos(lon), -np.sin(lat)*np.sin(lon), np.cos(lat)],
                     [-np.sin(lon), np.cos(lon),0],
                     [-np.cos(lat)*np.cos(lon), -np.cos(lat)*np.sin(lon), -np.sin(lat)]])
    return R_ned 
def gps_pos_vel(gps_ned, dt): 

    vel_ned = gps_ned / dt.values[:, np.newaxis]
    acc_ned = np.diff(vel_ned, axis=0) / dt.values[1:, np.newaxis]  
    displacement = np.cumsum(gps_ned, axis=0)
    return vel_ned, acc_ned, displacement

def ecef_to_ned(gps_pos_ref):
    deltas = (gps_pos_ref[['Node[1].ECEF.X', 'Node[1].ECEF.Y', 'Node[1].ECEF.Z']].values /1000) - (gps_pos_ref[['Node[1].ECEF.X.ref', 'Node[1].ECEF.Y.ref', 'Node[1].ECEF.Z.ref']].values/1000)   
    
    #use reference lat and lon points 
    lons = gps_pos_ref['Node[1].LLH.Lon.ref'] * (np.pi/180) 
    lats = gps_pos_ref['Node[1].LLH.Lat.ref'] * (np.pi/180)

    ned_values = []
    mags = []
    timestamps = []
    for i, (lat, lon) in enumerate(zip(lats, lons)): 
        rotation_matrix = rotation_ned(lat,lon)
        norm = np.linalg.norm(deltas[i,:])
        ned_vec = rotation_matrix @ deltas[i,:] / norm
        ned_mag = np.linalg.norm(ned_vec)
        mags.append(ned_mag )
        ned_values.append(ned_vec * norm)
        timestamps.append(gps_pos_ref.iloc[i]['timestamp'])
    ned_values = np.array(ned_values)
    return ned_values, np.array(mags), np.array(timestamps)

new_gps = gps_df.copy()
new_gps[['Node[1].ECEF.X.ref','Node[1].ECEF.Y.ref', 'Node[1].ECEF.Z.ref',
         'Node[1].LLH.Lat.ref','Node[1].LLH.Lon.ref','timestamp_ref']] = new_gps[["Node[1].ECEF.X", "Node[1].ECEF.Y",
                                                                                  "Node[1].ECEF.Z",'Node[1].LLH.Lat',
                                                                                  'Node[1].LLH.Lon', 'timestamp']].shift(1)

gps_pos_ref = new_gps[["Node[1].ECEF.X", "Node[1].ECEF.Y",
                       "Node[1].ECEF.Z",'Node[1].ECEF.X.ref',
                       'Node[1].ECEF.Y.ref', 'Node[1].ECEF.Z.ref',
                      'Node[1].LLH.Lat.ref', 'Node[1].LLH.Lon.ref',
                      'timestamp','timestamp_ref','interval_id']].dropna()

gps_ned, ned_mags, timestamps = ecef_to_ned(gps_pos_ref) 
print(gps_ned.shape, timestamps.shape)

dt = gps_pos_ref['timestamp'] - gps_pos_ref['timestamp_ref']
vel_ned, acc_ned, displacement = gps_pos_vel(gps_ned, dt)
print(vel_ned.shape, acc_ned.shape, displacement.shape)
gps_vel_pos = np.column_stack([vel_ned, displacement, timestamps])

gps_vel_pos = pd.DataFrame(gps_vel_pos, columns=['vel.x', 'vel.y', 'vel.z', 'pos.x', 'pos.y', 'pos.z', 'timestamp'])
gps_vel_pos.head()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 10))

# Plot velocity (left column)
axes[0, 0].plot(gps_vel_pos['vel.x'])
axes[0, 0].set_title('Velocity X')

axes[1, 0].plot(gps_vel_pos['vel.y'])
axes[1, 0].set_title('Velocity Y')

axes[2, 0].plot(gps_vel_pos['vel.z'])
axes[2, 0].set_title('Velocity Z')

# Plot position (right column)
axes[0, 1].plot(gps_vel_pos['pos.x'])
axes[0, 1].set_title('Position X')

axes[1, 1].plot(gps_vel_pos['pos.y'])
axes[1, 1].set_title('Position Y')

axes[2, 1].plot(gps_vel_pos['pos.z'])
axes[2, 1].set_title('Position Z')

# Optional: Improve layout
for ax in axes.flat:
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def get_acc(row):
    acc = np.sqrt(row['xl.x']**2 + row['xl.y']**2)
    if row['xl.x'] < 0 :
        acc*=-1

    return acc

def xl_state_estimate(x_prev, u, F, dt):
    x_k = F @ np.array(x_prev) + np.array([(dt**2)/2, dt]) * u 
    return x_k 


def Kalman_acc_update(x_prev, P_prev, Q_xl, F, row, imu_dt):

    acc = get_acc(row)
    x_update = xl_state_estimate(x_prev, acc, F, imu_dt)
    P_update = F @ P_prev @ F.T + Q_xl

    return x_update, P_update

def Kalman_gps_udpate(gps_state, P_prev, x_prev, R_gps):
    
    K = P_prev @ np.linalg.inv(P_prev + R_gps)
    H = np.array([[1, 0], [0, 1]])  #

    x_update = x_prev + K @ (gps_state - x_prev)
    # P_update = P_prev - K @ Q_gps
    # P_update = (np.eye(2) - K @ H) @ P_prev
    I = np.eye(2)
    P_update = (I - K @ H) @ P_prev @ (I - K @ H).T + K @ R_gps @ K.T

    # print("GPS UPDATE", x_update)
    return x_update, P_update, K

    
def test_kalman(xl_global, gps_df, init_x, gps_dt):
    
    gps_idx = 0 
    x_history = [init_x]
    P_history = []
    gain_history = []

    spectral_density_variance = 0.0001 
    velocity_random_walk_variance_xy =  0.00001 

    for i, row in xl_global.iterrows():

        if i == 0:
            imu_dt = .001
            gps_dt = .2


            Q_xl = spectral_density_variance * np.array([
                    [imu_dt**4 / 4, imu_dt**3 / 2],
                    [imu_dt**3 / 2, imu_dt**2]
                ]) + velocity_random_walk_variance_xy * np.array([
                    [0, 0],
                    [0, 1]
                ])

            P_history.append(Q_xl)
        else: 
            imu_dt = xl_global.iloc[i]['timestamp_aligned'] - xl_global.iloc[i-1]['timestamp_aligned']
            gps_dt = gps_df.iloc[gps_idx]['timestamp'] - gps_df.iloc[gps_idx-1]['timestamp']

        F = np.array([[1, imu_dt],[0, 1]]) 

        
        Q_xl = spectral_density_variance * np.array([
                [imu_dt**4 / 4, imu_dt**3 / 2],
                [imu_dt**3 / 2, imu_dt**2]
            ]) + velocity_random_walk_variance_xy * np.array([
                [0, 0],
                [0, 1]
            ])
        
        gps_vel_var = (0.125 * 10) * gps_dt
        gps_pos_var = (gps_vel_var * 10) * gps_dt**2

        R_gps = np.array([[gps_pos_var,0],[0, gps_vel_var]]) 

        xl_ts = row['timestamp_aligned']
        match = gps_df[round(gps_df['timestamp'],1) == round(xl_ts,3)]
                    
        x_k, P_k = Kalman_acc_update(x_history[-1], P_history[-1], Q_xl, F, row, imu_dt)
        # print("X_K", x_k)

        if not match.empty and xl_ts != 46757790.0:
            gps_idx += 1
            vel_xyz = match[['vel.x', 'vel.y']]
            pos_xyz = match[['pos.x', 'pos.y']]
            gps_vel = np.linalg.norm(vel_xyz, axis=1)
            gps_pos = np.linalg.norm(pos_xyz, axis=1)
            gps_state = np.array([gps_pos[0], gps_vel[0]])
            # print("GPS STATE,XPREV",gps_state,  x_history[-1])

            x_update, P_update, gain = Kalman_gps_udpate(gps_state, P_k, x_k, R_gps)

            x_history.append(x_update)
            P_history.append(P_update)
            gain_history.append(gain)

        else:
            x_history.append(x_k)
            P_history.append(P_k)

        if i == 100000:
            break
    return np.array(x_history), np.array(P_history), gain_history


In [ ]:
inital_pos = 0
inital_vel = 0
init_x = np.array([inital_pos, inital_vel])
centered = xl_static_df[['xl.x', 'xl.y']] - xl_static_df[['xl.x', 'xl.y']].mean()
centered['timestamp_aligned'] = xl_static_df['timestamp_aligned']

x_history, P_history, gain_history = test_kalman(centered, gps_vel_pos, init_x, gps_dt=0.2)

In [ ]:
def imu_dead_reckoning(xl_ned, prev_pos, prev_vel, dt):
    
    velocity = np.empty((len(xl_ned), 3))
    position = np.empty((len(xl_ned), 3))

    if prev_vel is not None: 
        velocity[0] = prev_vel
    if prev_pos is not None: 
        position[0] = prev_pos

    start_idx = 1 if prev_vel is not None and prev_pos is not None else 0

    for t in range(start_idx, len(xl_ned)):
        velocity[t] = velocity[t-1] + xl_ned[t-1] * dt
        position[t] = position[t-1] + velocity[t] * dt

    curr_pos = position[-1,:]
    curr_vel = velocity[-1,:]
    return np.concatenate([curr_pos, curr_vel])

def ewma(xl_global, gps_df, init_x, alpha):

    x_history = [init_x]
    for i, row in xl_global.iterrows():
            
        state_prev = x_history[-1]
        prev_pos = state_prev[0]
        prev_vel = state_prev[1]

        # vel_update = prev_vel + (acc * xl_dt)
        # pos_update = prev_pos + (vel_update * xl_dt)
        
        if i != 0: 
            dt = xl_global.loc[i,'timestamp_aligned'] - xl_global.loc[i - 1,'timestamp_aligned']
        else:
            dt = 1/1000

        xl_ts = row['timestamp_aligned']
        match = gps_df[round(gps_df['timestamp'],1) == round(xl_ts,1)]
        
        if xl_ts > gps_df.iloc[-1]['timestamp']:
            print('Exceeded gps timestamps')
            break

        if not match.empty and i != 0:
            vel_xyz = match[['vel.x', 'vel.y', 'vel.z']].values.flatten()
            pos_xyz = match[['pos.x', 'pos.y', 'pos.z']].values.flatten()

            gps_state = np.concatenate([pos_xyz, vel_xyz])
            state_pred = x_history[-1]
            # print(i)
            # print('state_prev',state_prev)
            # print('gps_state',gps_state)
            gps_update = state_pred + alpha * (gps_state - state_pred) 
            # gps_update = (1-alpha) * state_pred + alpha * (gps_state)
            # print('gps_update',gps_update)
            x_history.append(gps_update)

        else:
            prev_state = x_history[-1]  
            prev_pos = prev_state[:3] 
            prev_vel = prev_state[3:]
            
            xl_ned = row[['xl.x','xl.y','xl.z']].values
            state_update = imu_dead_reckoning(xl_ned, prev_pos, prev_vel, dt)
            x_history.append(state_update)

        if i == 30000:
            break
    return np.array(x_history)

def double_ewma(xl_global, gps_df,init_x, alpha):

    s_prime_history = [init_x]
    s_double_prime_history = [init_x]
    x_history = [init_x]

    for i,row in xl_global.iterrows():


        xl_ts = row['timestamp_aligned']
        match = gps_df[round(gps_df['timestamp'],1) == round(xl_ts,3)]

        if i != 0: 
            dt = xl_global.loc[i,'timestamp_aligned'] - xl_global.loc[i - 1,'timestamp_aligned']
        else:
            dt = 1/1000

        if not match.empty and i != 0:

            gps_vel = match[['vel.x', 'vel.y', 'vel.z']].values.flatten()
            gps_pos = match[['pos.x', 'pos.y', 'pos.z']].values.flatten()
            gps_state = np.concatenate([gps_pos, gps_vel])

            s_prime_prev = s_prime_history[-1]
            s_double_prev = s_double_prime_history[-1]

            s_prime_update = (alpha * gps_state) + (1-alpha) * s_prime_prev
            s_double_update = (alpha * s_prime_update) + (1-alpha) * s_double_prev
            
            s_prime_history.append(s_prime_update)
            s_double_prime_history.append(s_double_update)

            x_history.append((2*s_prime_update) - s_double_update)

        else:
            prev_state = x_history[-1]  
            prev_pos = prev_state[:3] 
            prev_vel = prev_state[3:]
            
            xl_ned = row[['xl.x','xl.y','xl.z']].values
            state_update = imu_dead_reckoning(xl_ned, prev_pos, prev_vel, dt)
            x_history.append(state_update)


        if i == 30000:
            break
    return np.array(x_history)

In [ ]:
ewma_df = pd.DataFrame(ewma_history, columns=['pos.x','pos.y','pos.z','vel.x','vel.y','vel.z'])
fig, axes = plt.subplots(3, 2, figsize=(12, 10))

# Plot velocity (left column)
axes[0, 0].plot(ewma_df['vel.x'])
axes[0, 0].plot(ewma_df['vel.x'].rolling(window=500).mean(), label='Avg')
axes[0, 0].set_title('Velocity X')


axes[1, 0].plot(ewma_df['vel.y'])
axes[1, 0].plot(ewma_df['vel.y'].rolling(window=300).mean(), label='Avg')
axes[1, 0].set_title('Velocity Y')

axes[2, 0].plot(ewma_df['vel.z'])
axes[2, 0].plot(ewma_df['vel.z'].rolling(window=300).mean(), label='Avg')
axes[2, 0].set_title('Velocity Z')

# Plot position (right column)
axes[0, 1].plot(ewma_df['pos.x'])
axes[0, 1].plot(ewma_df['pos.x'].rolling(window=300).mean(), label='Avg')
axes[0, 1].set_title('Position X')

axes[1, 1].plot(ewma_df['pos.y'])
axes[1, 1].plot(ewma_df['pos.y'].rolling(window=300).mean(), label='Avg')
axes[1, 1].set_title('Position Y')

axes[2, 1].plot(ewma_df['pos.z'])
axes[2, 1].plot(ewma_df['pos.z'].rolling(window=300).mean(), label='Avg')
axes[2, 1].set_title('Position Z')

# Optional: Improve layout
for ax in axes.flat:
    ax.grid(True)
    ax.legend()

fig.suptitle('Fusion IMU + GPS EWMA')
plt.tight_layout()
plt.show()